<a href="https://colab.research.google.com/github/revy1817/NLP_BG/blob/main/HomeWork_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [179]:
!pip install pymorphy2 stop_words slovnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49 kB 2.7 MB/s 


In [1]:
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 3.2 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [71]:
import pandas as pd
import re

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from tensorflow.keras.utils import to_categorical
#from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [224]:
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Учеба/GeekBrains/Введение в обработку естественного языка/Less 7/отзывы за лето.xls")
df

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
...,...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01
20655,5,Ок,2017-06-01
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01


In [223]:
max_words = 10000
max_len = 40
num_classes = 1

# Training
epochs = 4
batch_size = 512
print_batch_n = 100

In [225]:
# обработем текст + разделим таргет с мультикласса на оценки 1-3 отрицательные с 4-5 положительные
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

df['text'] = df['Content'].apply(preprocess_text)
df['target'] = (df['Rating'] > 3)*1

In [226]:
df['target'] = df['target'].astype(int)
df.head()

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,в целое удобноной приложениеиз минус хотеть сл...,1
2,5,Отлично все,2017-08-14,отлично всё,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать на 1 работа антивирус далёкий н...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро,1


In [227]:
df.drop(['Content', 'Date'], axis=1, inplace=True)

In [228]:
train_corpus = " ".join(df["text"])
train_corpus = train_corpus.lower()

In [229]:
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [230]:
tokens_filtered = [word for word in tokens if word.isalnum()]
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [231]:
tokens_filtered_top[:10]

['приложение', 'всё', 'и', 'очень', 'удобно', 'в', 'я', 'на', 'работать', 'с']

In [232]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [233]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [234]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.3,
                                                    random_state=13, stratify=df['target'])

In [235]:
X_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
X_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [236]:
X_train.shape

(14461, 40)

In [237]:
num_classes = 2
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [238]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [239]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [240]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=[X_test, y_test],
                    callbacks=[tensorboard])

Epoch 1/4
29/29 [==============================] - 6s 188ms/step - loss: 0.6087 - accuracy: 0.7775 - val_loss: 0.4594 - val_accuracy: 0.8141
Epoch 2/4
29/29 [==============================] - 5s 183ms/step - loss: 0.3601 - accuracy: 0.8389 - val_loss: 0.2619 - val_accuracy: 0.8906
Epoch 3/4
29/29 [==============================] - 5s 181ms/step - loss: 0.2211 - accuracy: 0.9062 - val_loss: 0.2183 - val_accuracy: 0.9029
Epoch 4/4
29/29 [==============================] - 5s 183ms/step - loss: 0.1732 - accuracy: 0.9306 - val_loss: 0.2072 - val_accuracy: 0.9105


**Задание №2:** Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

Будем использовать navec из наташи

In [174]:
! pip install navec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [175]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2022-07-31 16:50:13--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  15.6MB/s    in 3.2s    

2022-07-31 16:50:17 (15.6 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [241]:
from navec import Navec

path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [242]:
embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in vocabulary.items():
    embedding_vector = navec.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 4850 words (5149 misses)


In [243]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix),trainable=False, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [244]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [245]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=[X_test, y_test],
                    callbacks=[tensorboard])

Epoch 1/4
29/29 [==============================] - 8s 255ms/step - loss: 0.3388 - accuracy: 0.8533 - val_loss: 0.2463 - val_accuracy: 0.9014
Epoch 2/4
29/29 [==============================] - 7s 254ms/step - loss: 0.2216 - accuracy: 0.9095 - val_loss: 0.2335 - val_accuracy: 0.9050
Epoch 3/4
29/29 [==============================] - 7s 247ms/step - loss: 0.1899 - accuracy: 0.9282 - val_loss: 0.2326 - val_accuracy: 0.8979
Epoch 4/4
29/29 [==============================] - 7s 246ms/step - loss: 0.1630 - accuracy: 0.9431 - val_loss: 0.2244 - val_accuracy: 0.9064


Попробуем также использовать предобученный эмбендинг, но сделаем этот слой с возможностью обучения

In [246]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [247]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [248]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=[X_test, y_test],
                    callbacks=[tensorboard])

Epoch 1/4
29/29 [==============================] - 17s 556ms/step - loss: 0.3272 - accuracy: 0.8462 - val_loss: 0.2348 - val_accuracy: 0.8993
Epoch 2/4
29/29 [==============================] - 16s 549ms/step - loss: 0.2088 - accuracy: 0.9109 - val_loss: 0.2171 - val_accuracy: 0.9076
Epoch 3/4
29/29 [==============================] - 16s 555ms/step - loss: 0.1763 - accuracy: 0.9297 - val_loss: 0.2100 - val_accuracy: 0.9109
Epoch 4/4
29/29 [==============================] - 17s 605ms/step - loss: 0.1504 - accuracy: 0.9446 - val_loss: 0.2076 - val_accuracy: 0.9124


**Вывод:** На первых эпохах модель с предобученными весами показывает результат выше, но со временем модель которая сама задает веса показывает результат лучше, а вот модель с предобученными весами и возможностью переобучения весов эмбенинга показала самый наилучший результат